In [ ]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Import the requests library.
import requests
import gmaps
import requests

# Import the API key.
from config_google import g_key
%matplotlib inline

In [ ]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

In [ ]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [ ]:
# Heatmap of temperature

city_data_df = city_data_df.loc[(city_data_df['Max Temp'] > 0)]

# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"] 

# Assign the figure variable.
fig = gmaps.figure()
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

heat_layer = gmaps.heatmap_layer(locations, weights=max_temp)

In [ ]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

heat_layer = gmaps.heatmap_layer(locations, weights=temps)

In [ ]:
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp], dissipating=False, max_intensity=300, point_radius=4)

In [ ]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

In [ ]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)


In [ ]:
preferred_cities_df.count()

In [19]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Vaini,TO,86.16,-21.2000,-175.2000,
4,Hilo,US,82.13,19.7297,-155.0900,
7,Cabo San Lucas,MX,75.51,22.8909,-109.9124,
10,Carnarvon,AU,75.27,-24.8667,113.6333,
11,Poum,NC,82.27,-20.2333,164.0167,
12,Atuona,PF,78.98,-9.8000,-139.0333,
20,Namatanai,PG,85.62,-3.6667,152.4333,
21,Edd,ER,77.94,13.9297,41.6925,
24,Rikitea,PF,80.38,-23.1203,-134.9692,
28,Alofi,NU,84.09,-19.0595,-169.9187,


In [20]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [25]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        hotel_df.loc[index, "Hotel Name"] = "Not Found"

hotel_df

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Vaini,TO,86.16,-21.2000,-175.2000,Keleti Beach Resort
4,Hilo,US,82.13,19.7297,-155.0900,Hilo Hawaiian Hotel
7,Cabo San Lucas,MX,75.51,22.8909,-109.9124,Hotel Tesoro Los Cabos
10,Carnarvon,AU,75.27,-24.8667,113.6333,Hospitality Carnarvon
11,Poum,NC,82.27,-20.2333,164.0167,Not Found
...,...,...,...,...,...,...
532,Beira,MZ,77.09,-19.8436,34.8389,Beira Terrace Hotel
533,Noumea,NC,85.98,-22.2763,166.4572,Hôtel Le Lagon
539,La Asuncion,VE,76.15,11.0333,-63.8628,Colinas de Matasiete
540,Omboue,GA,79.00,-1.5746,9.2618,Hotel Olako


In [26]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [28]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [31]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [34]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
hotel_info


['\n<dl>\n<dt>Hotel Name</dt><dd>Keleti Beach Resort</dd>\n<dt>City</dt><dd>Vaini</dd>\n<dt>Country</dt><dd>TO</dd>\n<dt>Max Temp</dt><dd>86.16 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Hilo Hawaiian Hotel</dd>\n<dt>City</dt><dd>Hilo</dd>\n<dt>Country</dt><dd>US</dd>\n<dt>Max Temp</dt><dd>82.13 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Hotel Tesoro Los Cabos</dd>\n<dt>City</dt><dd>Cabo San Lucas</dd>\n<dt>Country</dt><dd>MX</dd>\n<dt>Max Temp</dt><dd>75.51 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Hospitality Carnarvon</dd>\n<dt>City</dt><dd>Carnarvon</dd>\n<dt>Country</dt><dd>AU</dd>\n<dt>Max Temp</dt><dd>75.27 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Not Found</dd>\n<dt>City</dt><dd>Poum</dd>\n<dt>Country</dt><dd>NC</dd>\n<dt>Max Temp</dt><dd>82.27 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Villa Enata</dd>\n<dt>City</dt><dd>Atuona</dd>\n<dt>Country</dt><dd>PF</dd>\n<dt>Max Temp</dt><dd>78.98 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Nam

In [33]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))